In [1]:
pip install openai -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import json
from IPython.display import display, Image, Markdown
from openai import OpenAI
import pandas as pd
from pprint import pprint

In [3]:
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

In [4]:
client = OpenAI(api_key= get_secret('openai')['api_key'])

In [10]:
df = pd.read_csv('CrimeRate.csv')
df_sample = df.sample(50, random_state=1)
df_sample = df_sample.fillna("").astype(str)
data_json = df_sample.to_json(orient="records")
df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,200412582,09/09/2020 12:00:00 AM,09/09/2020 12:00:00 AM,630,4,Hollenbeck,413,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,200 E AVENUE 28,NaN,34.0820,-118.2130


In [11]:
data_prompt = f"Explain how insights from this data could help local law enforcement or city officials make decisions about where to allocate resources, increase patrols, or investigate root causes. What ethical concerns should be considered when analyzing and acting on this data? Data: {data_json}"
# print(prompt)

In [12]:
def openai_gpt_help(prompt):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=messages,
        temperature = 0
    )
    token_usage = response.usage
    
    pprint(f"Tokens used: {token_usage}")

    return response.choices[0].message.content

In [13]:
gpt_result = openai_gpt_help(prompt=data_prompt)

('Tokens used: CompletionUsage(completion_tokens=618, prompt_tokens=11200, '
 'total_tokens=11818, '
 'completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, '
 'audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), '
 'prompt_tokens_details=PromptTokensDetails(audio_tokens=0, '
 'cached_tokens=11136))')


In [14]:
display(Markdown(gpt_result))

Analyzing crime data can provide valuable insights for local law enforcement and city officials to make informed decisions about resource allocation, patrol increases, and investigating root causes of crime. Here’s how insights from the provided data can be utilized:

### Resource Allocation and Patrol Increases
1. **Identifying Hotspots**: By mapping the locations of reported crimes, officials can identify hotspots where crime is more prevalent. For instance, if a particular area, such as "Rampart" or "Van Nuys," shows a high frequency of vehicle thefts or assaults, resources can be allocated to increase patrols in these areas.

2. **Time-Based Patterns**: Analyzing the time of day when crimes occur can help in scheduling patrols more effectively. For example, if data shows a spike in burglaries during late night or early morning hours, patrols can be increased during these times.

3. **Crime Type Analysis**: Understanding the types of crimes that are most common in certain areas can help in deploying specialized units. For instance, areas with high rates of aggravated assault might benefit from increased presence of officers trained in de-escalation techniques.

### Investigating Root Causes
1. **Demographic Analysis**: Examining the demographics of victims and perpetrators can provide insights into potential social issues or tensions that may be contributing to crime. This can guide community outreach and intervention programs.

2. **Environmental Factors**: Analyzing the premises where crimes occur (e.g., streets, multi-unit dwellings) can help identify environmental factors that may contribute to crime, such as poor lighting or lack of surveillance.

3. **Repeat Offenses**: Identifying patterns of repeat offenses in certain areas can indicate systemic issues, such as lack of social services or economic opportunities, which may require long-term strategic planning and community engagement.

### Ethical Concerns
1. **Privacy and Data Security**: Ensuring that the data is anonymized and securely stored is crucial to protect the privacy of individuals involved. Sensitive information should not be publicly accessible or used in a way that could harm individuals.

2. **Bias and Discrimination**: Care must be taken to avoid reinforcing biases or discrimination. For example, over-policing in certain neighborhoods based on historical data can lead to racial profiling and community distrust.

3. **Transparency and Accountability**: Law enforcement agencies should be transparent about how data is used to make decisions. Engaging with community stakeholders and providing clear communication about the rationale for resource allocation can build trust.

4. **Data Interpretation**: Misinterpretation of data can lead to ineffective or harmful policies. It is important to involve data analysts and criminologists who can provide context and ensure that conclusions drawn from the data are accurate and actionable.

5. **Community Involvement**: Involving community members in discussions about crime prevention strategies can ensure that interventions are culturally sensitive and address the needs of the community.

By carefully analyzing crime data and considering these ethical concerns, law enforcement and city officials can make informed decisions that enhance public safety while maintaining community trust and respect for individual rights.

# Reflection
GPT-4o effectively identified patterns and explained how insights from the crime dataset could inform resource allocation and investigation strategies. The model structured the output clearly and included ethical concerns, which is important in real world applications of intelligence analysis. While the output was insightful, it required trimming the dataset due to token limits, which might omit useful context. Still, this shows great potential for GPT-4o to assist analysts during early stages of crime pattern recognition and policy planning.
